- Image scale normalization
- Image format normalization (.jpg, .png) ✅
- overlay ✅
- blurring ✅
- pixelate ✅
- sharpness
- shifting ✅
- merging?

In [335]:
# import dependencies

import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [336]:
# Image initialization

def _get_image(path):
    img = mimg.imread(path)
    return_img = np.zeros(shape=(img.shape[0], img.shape[1], 3))

    if path.endswith('.jpg'):
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                for k in range(3):
                    return_img[i, j, k] = img.item((i, j, k)) / 255
    return return_img


img1 = _get_image('Input Images/Spiral.jpg')
img2 = _get_image('Input Images/Oogway_forest.jpg')
img3 = _get_image('Input Images/F.png')

# img2 = np.zeros((1200, 1920, 3))
# for i in range(img2.shape[0]):
#     for j in range(img2.shape[1]):
#         for k in range(3):
#             img2[i, j, k] = 255

In [337]:
# dependency functions

# overlay dependency
def _avg_pixel_weighted(channel_val1, weight1, channel_val2, weight2):
    adjusted_weight_total = weight1 + weight2
    return ( (channel_val1 * weight1 + channel_val2 * weight2)/adjusted_weight_total)

def _find_stop_pixelate(height,  width, grid_size, start_pixel):
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    e_h = height % grid_size if (height - x_dim) <= padding else None # second axis 
    e_w = width % grid_size if (width - y_dim) <= padding else None # first axis

    empty_stop_height = -padding + e_h if e_h is not None else _stop
    empty_stop_width = -padding + e_w if e_w is not None else _stop      # refactor _stop to above

    return [empty_stop_height, empty_stop_width]

def _find_stop_blur(height, width, grid_size, start_pixel):
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    if x_dim < height or y_dim < width:
        e_h = height - x_dim if (height - x_dim) <= padding and (x_dim < height) else _stop
        e_w = width - y_dim if (width - y_dim) <= padding and (y_dim < width) else _stop

    if x_dim >= height or y_dim >= width:
        e_h = height - (x_dim - padding) if (height - x_dim) <= padding and (x_dim >= height) else _stop
        e_w = width - (y_dim - padding) if (width - y_dim) <= padding and (y_dim >= width) else _stop

    return [e_h, e_w]


# blur, pixelate dependency
def _avg_color_in_grid(img, grid_size, start_pixel, img_type):

    x_dim, y_dim = start_pixel
    padding = grid_size//2

    _stop = (padding + 1) if grid_size & 1 else padding
    try:

        r_total, g_total, b_total = 0, 0, 0
        total_pixels = grid_size**2

        for _i in range(-padding, _stop):
            for _j in range(-padding, _stop):

                r_total += img.item((x_dim + _i, y_dim + _j, 0))
                g_total += img.item((x_dim + _i, y_dim + _j, 1))
                b_total += img.item((x_dim + _i, y_dim + _j, 2))

    except (IndexError):
        r_total, g_total, b_total = 0, 0, 0

        if img_type == "BLUR":
            empty_stop_height, empty_stop_width = _find_stop_blur(img.shape[0], img.shape[1], grid_size, start_pixel)
        elif img_type == "PIXELATE":
            empty_stop_height, empty_stop_width = _find_stop_pixelate(img.shape[0], img.shape[1], grid_size, start_pixel)

        total_pixels = 0
        for _i in range(-padding, empty_stop_height):
            for _j in range(-padding, empty_stop_width):

                if (x_dim + _i >= 0) and (y_dim + _j >= 0):  # maybe x_dim + _i < height

                    r_total += img.item((x_dim + _i, y_dim + _j, 0))
                    g_total += img.item((x_dim + _i, y_dim + _j, 1))
                    b_total += img.item((x_dim + _i, y_dim + _j, 2))


                    total_pixels+=1

    return np.array((r_total / total_pixels, g_total / total_pixels, b_total / total_pixels))


In [338]:
# OVERLAY

def overlay(img1, img1_weight, img2, img2_weight):

    max_x_size = max(img1.shape[0], img2.shape[0])
    max_y_size = max(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(max_x_size, max_y_size, 3))

    for i in range(max_x_size):
        for j in range(max_y_size):
            for k in range(3):

                new_img_arr[i, j, k] = _avg_pixel_weighted(img1.item((i, j, k)), img1_weight, img2.item((i, j, k)), img2_weight)  # 4 sec

    return new_img_arr


In [339]:
# PIXELATE

def pixelate(img, pixelate_amount):

    height = img.shape[0]
    width = img.shape[1]

    padding = pixelate_amount//2

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(padding, height + padding, pixelate_amount):
        for j in range(padding, width + padding, pixelate_amount):
            color = _avg_color_in_grid(img, pixelate_amount, (i, j), "PIXELATE")

            try:

                for _i in range(-padding, (padding +1) if pixelate_amount & 1 else padding):
                    for _j in range(-padding, (padding +1) if pixelate_amount & 1 else padding):

                        new_img_arr[i + _i, j + _j] = color

            except(IndexError):
                empty_stop_height, empty_stop_width = _find_stop_pixelate(height, width, pixelate_amount, (i, j))

                for _i in range(-padding, empty_stop_height):
                    for _j in range(-padding, empty_stop_width):
                        
                        new_img_arr[i + _i, j + _j] = color

    return new_img_arr


In [340]:
# BLUR

def blur(img, blur_size = 3):
    # blur_size = int(input("Blur size: "))
    # 2 - 17.8s
    # 3 - 24.6s
    # 5 - ~47s
    # 7 - 1m 19.2s
    # 10 - 2m 37.5s

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.zeros(shape=(height, width, 3))

    # width-1 if blur_size & 1 else width
    for i in range(height):
        for j in range(width):

            new_img_arr[i, j] = _avg_color_in_grid(img, blur_size, (i, j), "BLUR")

    # last width is 1919

    return new_img_arr


In [347]:
# SHIFTING

def shift(img, operation, start_val_increment=1.6, shift_x=10, shift_y=10, start_offset=0):
    # operation = int(input("what operation should be performed? (0:linear, 1:sin, 2:cos, 3:tan, 4:random)  "))
    multiplicant = 0

    start_val = 0

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                if operation == 4:
                    multiplicant = random.random() * 2 - 1

                new_img_arr[i, j, k] = img.item((
                    ((int(multiplicant * shift_x)) + i + start_offset) %
                    height,
                    ((int(multiplicant * shift_y)) + j + start_offset) %
                    width, k))

        multiplicant = start_val if operation == 0 else math.sin(start_val) if operation == 1 else math.cos(start_val) if operation == 2 else math.tan(start_val) if operation == 3 else 0

        start_val+=start_val_increment # for sin or cos, min and max shift is from 0 to 1.5707 (π/2)

    return new_img_arr


In [351]:
# plt.imsave("Output Images/Overlay/Overlay 3-1.jpg", overlay(img1, img2))
# plt.imsave("Output Images/Blur/Blur num2hel.png", blur(img2, blur_size=3))
# plt.imsave("Output Images/Pixelate/Pixelate test25.png", pixelate(img2, 27))
# plt.imsave("Output Images/Distort/Random.png", shift(img2, 4))